In [29]:
import pandas as pd
bike_df=pd.read_csv('data/citibike-tripdata.csv', sep=',')

# инфо по признакам в датасете (кол-во строк - 1.7 миллионов)
# starttime — время начала поездки (дата, время);
# stoptime — время окончания поездки (дата, время);
# start station id — идентификатор стартовой стоянки;
# start station name — название стартовой стоянки;
# start station latitude, start station longitude — географическая широта и долгота стартовой стоянки;
# end station id — идентификатор конечной стоянки;
# end station name — название конечной стоянки;
# end station latitude, end station longitude — географическая широта и долгота конечной стоянки;
# bikeid — идентификатор велосипеда;
# usertype — тип пользователя (Customer — клиент с подпиской на 24 часа или на три дня, Subscriber — подписчик с годовой арендой велосипеда);
# birth year — год рождения клиента;
# gender — пол клиента (0 — неизвестный, 1 — мужчина, 2 — женщина)

display(bike_df.head())

# задание 1. Сколько пропусков в столбце start station id
bike_df.info()
a=299831
b=300000
print(b-a)

# задание 2. Какой тип данных имеют столбцы starttime и stoptime
# через инфо - object

# Задание 3. Найдите идентификатор самой популярной стартовой стоянки. Запишите идентификатор в виде целого числа.
display(bike_df['start station id'].mode())

# Задание 4. Велосипед с каким идентификатором является самым популярным?
display(bike_df['bikeid'].mode())

# Задание 5. Какой тип клиентов (столбец usertype) является преобладающим — Subscriber или Customer? 
# В качестве ответа запишите долю клиентов преобладающего типа среди общего количества клиентов. Ответ округлите до сотых
# Мое решение:
display(bike_df['usertype'].value_counts(normalize=True))
# Решение курса:
mode_usertype = bike_df['usertype'].mode()[0]
count_mode_user = bike_df[bike_df['usertype'] == mode_usertype].shape[0]
print(round(count_mode_user / bike_df.shape[0], 2))

# Задание 6. Кто больше занимается велоспортом — мужчины или женщины? В ответ запишите число поездок для той группы, у которой их больше.
# Мое решение:
display(bike_df['gender'].value_counts())
# Решение курса:
male_count = bike_df[bike_df['gender'] == 1].shape[0]
female_count = bike_df[bike_df['gender'] == 0].shape[0]
print(max([male_count, female_count]))

# Задание 7.1
# Число уникальных стартовых и конечных стоянок, которыми воспользовались клиенты, не равны друг другу (да/нет)
display(bike_df['start station id'].nunique())
display(bike_df['end station id'].nunique())
# Ответ - да (не равны)

# Задание 7.2
# В рассматриваемые дни минимальный возраст клиента составлял 10 лет (да/нет)
# НЕ РЕШИЛ


# Задание 7.3
# C Самой непопулярной стартовой стоянкой из тех, которыми воспользовались клиенты, является стоянка с названием "Eastern Pkwy & Washington Ave" (да/нет)
display(bike_df['start station name'].value_counts())
# Ответ - да (6 значений)


# Задание 7.4
# Наибольшее количество поездок завершается на стоянке под названием "Liberty Light Rail" (да/нет)
display(bike_df['end station name'].value_counts())
# Ответ - нет (1 значение)

# Задание 8.
# В первую очередь удалим лишнюю информацию из данных.
# В наших данных присутствуют столбцы, которые дублируют информацию друг о друге: это столбцы с идентификатором и названием стартовой и конечной стоянки.
# Удалите признаки идентификаторов стоянок. Сколько столбцов осталось?
bike_df.drop(['start station id'], axis=1, inplace=True)
bike_df.drop(['end station id'], axis=1, inplace=True)
print(bike_df.shape[1])
# Ответ 12

# Задание 9.
# Замените признак birth year на более понятный признак возраста клиента age. Годом отсчёта возраста выберите 2018 год. 
# Столбец birth year удалите из таблицы. Сколько поездок совершено клиентами старше 60 лет?
bike_df['age']=2018-bike_df['birth year']
bike_df.drop(['birth year'], axis=1, inplace=True)
print(bike_df[bike_df['age'] > 60].shape[0])

# Задание 10.
# Создайте признак длительности поездки trip duration. Для этого вычислите интервал времени между временем окончания поездки (stoptime) и её началом (starttime). 
# Сколько целых минут длилась поездка под индексом 3 в таблице?
bike_df['starttime']=pd.to_datetime(bike_df['starttime'], dayfirst=False)
bike_df['stoptime']=pd.to_datetime(bike_df['stoptime'], dayfirst=False)
bike_df['trip duration']=bike_df['stoptime']-bike_df['starttime']
bike_df.loc[3, ['trip duration']]

# Задание 11. 
# Создайте «признак-мигалку» weekend, который равен 1, если поездка начиналась в выходной день (суббота или воскресенье), и 0 — в противном случае. 
# Выясните, сколько поездок начиналось в выходные.
weekday=bike_df['starttime'].dt.dayofweek
bike_df['weekend']=weekday.apply(lambda x: 1 if x == 5 or x == 6 else 0)
print(bike_df['weekend'].sum())

# Задание 12.
# Создайте признак времени суток поездки time_of_day. Время суток будем определять из часа начала поездки. Условимся, что:
# поездка совершается ночью (night), если её час приходится на интервал от 0 (включительно) до 6 (включительно) часов;
# поездка совершается утром (morning), если её час приходится на интервал от 6 (не включительно) до 12 (включительно) часов;
# поездка совершается днём (day), если её час приходится на интервал от 12 (не включительно) до 18 (включительно) часов;
# поездка совершается вечером (evening), если её час приходится на интервал от 18 (не включительно) до 23 часов (включительно).
# Во сколько раз количество поездок, совершённых днём, больше, чем количество поездок, совёршенных ночью, за представленный в данных период времени? Ответ округлите до целых.
def to_time_of_day(time):
    if 0 <= time <= 6:
        return 'night'
    if 6 < time <= 12:
        return 'morning'
    if 12 < time <= 18:
        return 'day'
    if 18 < time <= 23:
        return 'evening'

bike_df['time_of_day']=bike_df['starttime'].dt.hour.apply(to_time_of_day)
day=bike_df[bike_df['time_of_day']== 'day'].shape[0]
night=bike_df[bike_df['time_of_day']=='night'].shape[0]
print(round(day/night))

,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,2018-09-01 00:00:05.2690,2018-09-01 00:27:20.6340,252.0,MacDougal St & Washington Sq,40.732264,-73.998522,366.0,Clinton Ave & Myrtle Ave,40.693261,-73.968896,25577,Subscriber,1980,1
1,2018-09-01 00:00:11.2810,2018-09-01 00:02:23.4810,314.0,Cadman Plaza West & Montague St,40.693830,-73.990539,3242.0,Schermerhorn St & Court St,40.691029,-73.991834,34377,Subscriber,1969,0
2,2018-09-01 00:00:20.6490,2018-09-01 00:55:58.5470,3142.0,1 Ave & E 62 St,40.761227,-73.960940,3384.0,Smith St & 3 St,40.678724,-73.995991,30496,Subscriber,1975,1
3,2018-09-01 00:00:21.7460,2018-09-01 00:07:38.5830,308.0,St James Pl & Oliver St,40.713079,-73.998512,3690.0,Park Pl & Church St,40.713342,-74.009355,28866,Subscriber,1984,2
4,2018-09-01 00:00:27.3150,2018-09-01 02:21:25.3080,345.0,W 13 St & 6 Ave,40.736494,-73.997044,380.0,W 4 St & 7 Ave S,40.734011,-74.002939,20943,Customer,1994,1


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 14 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   starttime                300000 non-null  object 
 1   stoptime                 300000 non-null  object 
 2   start station id         299831 non-null  float64
 3   start station name       299831 non-null  object 
 4   start station latitude   300000 non-null  float64
 5   start station longitude  300000 non-null  float64
 6   end station id           299831 non-null  float64
 7   end station name         299831 non-null  object 
 8   end station latitude     300000 non-null  float64
 9   end station longitude    300000 non-null  float64
 10  bikeid                   300000 non-null  int64  
 11  usertype                 300000 non-null  object 
 12  birth year               300000 non-null  int64  
 13  gender                   300000 non-null  int64  
dtypes: f

0    281.0
Name: start station id, dtype: float64

0    33887
Name: bikeid, dtype: int64

Subscriber    0.774007
Customer      0.225993
Name: usertype, dtype: float64

...

759

765

Grand Army Plaza & Central Park S    1928
Central Park S & 6 Ave               1909
Pershing Square North                1873
12 Ave & W 40 St                     1845
West St & Chambers St                1835
                                     ... 
NYCBS Depot - GOW                      15
Franklin Ave & Empire Blvd             11
Railroad Ave & Kay Ave                  9
47 Ave & Skillman Ave                   7
Eastern Pkwy & Washington Ave           6
Name: start station name, Length: 759, dtype: int64

West St & Chambers St          1963
12 Ave & W 40 St               1909
Pershing Square North          1850
Central Park S & 6 Ave         1828
E 17 St & Broadway             1804
                               ... 
Exchange Place                    3
Union St                          1
Warren St                         1
Montrose Ave & Bushwick Ave       1
Liberty Light Rail                1
Name: end station name, Length: 765, dtype: int64

12
11837
115135
9
